In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# import os
# os.chdir('/mnt/disk2/arshia.yousefinezhad/emotion_detection')
# os.getcwd()

In [1]:
import pandas as pd
import time
import torch
import numpy as np
import re
from transformers import BertConfig, BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
import torch.nn as nn
from tqdm import tqdm

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

max_len = 64
train_batch_size = 64
valid_batch_size = 64
test_batch_size = 64

epoch = 3
EEVERY_EPOCH = 1000
lr = 2e-5
CLIP = 0.0

In [3]:
# def label_ecoding(df):
#     df.emotion.replace([' عاشقانه', 'عاشقانه', 'عاشقانه '], 'عاشقانه', inplace=True)
#     df.emotion.replace(['خوشال','خوشحال','خوشحال ','خوشحالی'], 'خوشحال', inplace=True)
#     df.emotion.replace(['هییجان انگیز ',  'هیجانزده',  'هیجان زده ', 'هیجان زده', 'هیجان انگیز ', 'هیجان انگیز', 'هیجان',
#                         'شگفت زده', 'تعجب','متعجب','با تمام مشکلات خودمو کشیدم بالا و گوشمو تابوندم هر کسی رو محرم خودم ندونم+D23901:D23906'],
#                         'هیجانی و متعجب', inplace=True)
#     df.emotion.replace(['نارحت', 'ناراحتی', 'ناراحت ', 'ناراحت', 'مناراحت', 'غمگین','نگران'], 'غمگین', inplace=True)
#     df.emotion.replace(['مضطرب','اضطراب'], 'مضطرب', inplace=True)
#     df.emotion.replace(['معمول','معمولا','معمولب','معمولی','معمولی ','معولی','ممعمولی','ممولی'], 'معمولی', inplace=True)
#     df.emotion.replace([ 'عصبانی','عصبانی ','اعصبانی','خشم', 'اعتراض', 'اعتراضی', 'تنفر'], 'عصبانی', inplace=True)
#     df.emotion.replace(['ترس','ترس ','ترسناک'], 'ترس', inplace=True)

#     df = df[~df['emotion'].isin(['رس','شرمندگی', 'نترس', 'اعتراضی'])]
#     return df

# def preprocess_text(df):
#     def process_text(text):
#       text = re.sub(r'[^ا-ی!\s]', '', text)
#       text = re.sub('[0-9]','',text)
#       return text
#     df.combined_text = df.combined_text.apply(process_text)
#     return df
data = pd.read_csv('/content/drive/MyDrive/#University_Tehran/train_process.csv')
# data_prepared = preprocess_text(data)

# data_prepared_labels = label_ecoding(data_prepared)
# data = data_prepared_labels.copy()

# Data Loader File

In [4]:
class CallCenterDataset(torch.utils.data.Dataset):
    def __init__(self, tokenizer, contexts, targets=None, label_list=None, max_len=128):
        self.contexts = contexts
        self.targets = targets
        self.has_target = isinstance(targets, list) or isinstance(targets, np.ndarray)
        self.tokenizer = tokenizer
        self.max_len = max_len

        # Create a label-to-index mapping if label_list is provided
        self.label_map = {label: i for i, label in enumerate(label_list)} if isinstance(label_list, list) else {}

    def __len__(self):
        return len(self.contexts)

    def __getitem__(self, item):
        context = str(self.contexts[item])
        if self.has_target:
            target = self.label_map.get(str(self.targets[item]), str(self.targets[item]))
        encoding = self.tokenizer.encode_plus(
            context,
            add_special_tokens=True,
            truncation=True,
            max_length=self.max_len,
            return_token_type_ids=True,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt')

        inputs = {
            'context': context,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'token_type_ids': encoding['token_type_ids'].flatten(),
        }
        if self.has_target:
            inputs['targets'] = torch.tensor(target, dtype=torch.long)
        return inputs

def create_data_loader(x, y, tokenizer, max_len, batch_size, label_list):
    dataset = CallCenterDataset(
        contexts=x,
        targets=y,
        tokenizer=tokenizer,
        max_len=max_len,
        label_list=label_list)
    return torch.utils.data.DataLoader(dataset, batch_size=batch_size, drop_last=True)

def split_data(df):
    labels_list = list(sorted(df['label'].unique()))
    df['label_id'] = df['label'].apply(lambda t: labels_list.index(t))
    train, test = train_test_split(df, test_size=0.1, random_state=1, stratify=df['label'])
    train, valid = train_test_split(train, test_size=0.1, random_state=1, stratify=train['label'])
    train = train.reset_index(drop=True)
    valid = valid.reset_index(drop=True)
    test = test.reset_index(drop=True)
    return train, valid, test


train, valid, test = split_data(data)

x_train, y_train = train['text'].values.tolist(), train['label_id'].values.tolist()
x_valid, y_valid = valid['text'].values.tolist(), valid['label_id'].values.tolist()
x_test, y_test = test['text'].values.tolist(), test['label_id'].values.tolist()

# Models

## Configurations

In [5]:
parsbert_id = 'HooshvareLab/bert-fa-base-uncased'

label_list = list(sorted(data['label'].unique()))

label2id = {label: i for i, label in enumerate(label_list)}
id2label = {v: k for k, v in label2id.items()}

## ParsBert

### Tokenization

In [6]:
tokenizer = BertTokenizer.from_pretrained(parsbert_id)
bertcofig = BertConfig.from_pretrained(
    parsbert_id, **{
        'label2id': label2id,
        'id2label': id2label,
    })

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


### create data loader

In [7]:
train_data_loader = create_data_loader(train['text'].to_numpy(), train['label'].to_numpy(), tokenizer, max_len, train_batch_size, label_list)
valid_data_loader = create_data_loader(valid['text'].to_numpy(), valid['label'].to_numpy(), tokenizer, max_len, valid_batch_size, label_list)
test_data_loader = create_data_loader(test['text'].to_numpy(), test['label'].to_numpy(), tokenizer, max_len, test_batch_size, label_list)

### Models artitecture

In [8]:
class TextClassificationLSTMBidirectional(nn.Module):
    def __init__(self, config, embedding_dim, hidden_dim, output_dim, n_layers,
                 bidirectional, dropout):
        super(TextClassificationLSTMBidirectional, self).__init__()
        self.bert = BertModel.from_pretrained('HooshvareLab/bert-fa-base-uncased')
        self.lstm = nn.LSTM(embedding_dim,
                           hidden_dim,
                           num_layers=n_layers,
                           bidirectional=bidirectional,
                           dropout=dropout,
                           batch_first=True)
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input_ids, attention_mask=None, token_type_ids=None):
        bert_output = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask,
                            token_type_ids=token_type_ids)
        # bert_output.last_hidden_state.size() = [batch_size, sen_len, 768]
        embedded = bert_output.last_hidden_state.permute(1, 0, 2)
        # embedded.size() =  [sen_len, batch_size, 768]
        output, (hidden, cell) = self.lstm(embedded)
        #output = [batch size, sent len, hid dim * num directions]
        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]
        #concat the final forward and backward hidden state
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))
        #hidden = [batch size, hid dim * num directions]
        return self.fc(hidden)

### Defining Training Process

In [9]:
# embedding_dim =
hidden_dim = 128
output_dim = len(label2id)
pad_idx = 0
n_epochs = 10
lr = 1e-1

EMBEDDING_DIM = 100
HIDDEN_DIM = 128
OUTPUT_DIM = len(label2id)
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5



model = TextClassificationLSTMBidirectional(
                  bertcofig,
                  EMBEDDING_DIM,
                  HIDDEN_DIM,
                  OUTPUT_DIM,
                  N_LAYERS,
                  BIDIRECTIONAL,
                  DROPOUT).to(device)

model = model.to(device)

def train(dataloader):
    model.train()
    total_acc, total_count, total_loss = 0, 0, 0
    log_interval = 500
    start_time = time.time()

    for batch_index, batch_data in enumerate(tqdm(dataloader)):

        # Extract input features and labels from the batch_data dictionary
        input_ids = batch_data['input_ids'].to(device)
        attention_mask = batch_data['attention_mask'].to(device)
        token_type_ids = batch_data['token_type_ids'].to(device)
        targets = batch_data['targets'].to(device)

        optimizer.zero_grad()
        predicted_targets = model(input_ids, attention_mask, token_type_ids)

        loss = criterion(predicted_targets, targets)
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)

        optimizer.step()

        total_acc += (predicted_targets.argmax(1) == targets).sum().item()
        total_count += targets.size(0)

        if batch_index % log_interval == 0 and batch_index > 0:
            elapsed = time.time() - start_time
            print(
                "| epoch {:3d} | {:5d}/{:5d} batches "
                "| accuracy {:8.3f}".format(
                    epoch, batch_index, len(dataloader), total_acc / total_count
                )
            )
            total_acc, total_count = 0, 0
            start_time = time.time()


def evaluate(dataloader):
  model.eval()
  total_acc, total_count = 0, 0

  with torch.no_grad():
    for batch_index, batch_data in enumerate(tqdm(dataloader)):
        input_ids = batch_data['input_ids'].to(device)
        attention_mask = batch_data['attention_mask'].to(device)
        targets = batch_data['targets'].to(device)

        predicted_targets = model(input_ids, attention_mask)

        loss = criterion(predicted_targets, targets)
        total_acc += (predicted_targets.argmax(1) == targets).sum().item()
        total_count += targets.size(0)
  return total_acc / total_count

### Training ...

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None

for epoch in tqdm(range(1, 10 + 1), desc="Epochs... "):
    epoch_start_time = time.time()
    train(train_data_loader)
    accu_val = evaluate(valid_data_loader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
      total_accu = accu_val
    print("-" * 59)
    print(
        "| end of epoch {:3d} | time: {:5.2f}s | "
        "valid accuracy {:8.3f} ".format(
            epoch, time.time() - epoch_start_time, accu_val
        )
    )
    print("-" * 59)

  0%|          | 0/77 [00:00<?, ?it/s]

### Save pretrained model

In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/#Part/Emotion_Detection/emotion_detection_parsbert.pt')

print("Checking the results of test dataset.")
accu_test = evaluate(test_data_loader)
print("test accuracy {:8.3f}".format(accu_test))

evaluate(train_data_loader)